ViT_LSTM_att

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Dec  5 16:21:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp -av '/content/drive/MyDrive/coco/pycocotools' '/content/'

import os
os.chdir('/content/drive/MyDrive/YOLOv4_CNN_RNN')

'/content/drive/MyDrive/coco/pycocotools' -> '/content/pycocotools'
'/content/drive/MyDrive/coco/pycocotools/_mask.pyx' -> '/content/pycocotools/_mask.pyx'
'/content/drive/MyDrive/coco/pycocotools/__init__.py' -> '/content/pycocotools/__init__.py'
'/content/drive/MyDrive/coco/pycocotools/_mask.c' -> '/content/pycocotools/_mask.c'
'/content/drive/MyDrive/coco/pycocotools/_mask.cp39-win_amd64.pyd' -> '/content/pycocotools/_mask.cp39-win_amd64.pyd'
'/content/drive/MyDrive/coco/pycocotools/mask.py' -> '/content/pycocotools/mask.py'
'/content/drive/MyDrive/coco/pycocotools/coco.py' -> '/content/pycocotools/coco.py'
'/content/drive/MyDrive/coco/pycocotools/__pycache__' -> '/content/pycocotools/__pycache__'
'/content/drive/MyDrive/coco/pycocotools/__pycache__/coco.cpython-39.pyc' -> '/content/pycocotools/__pycache__/coco.cpython-39.pyc'
'/content/drive/MyDrive/coco/pycocotools/__pycache__/__init__.cpython-39.pyc' -> '/content/pycocotools/__pycache__/__init__.cpython-39.pyc'
'/content/drive/My

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # hadie
# You'll generate plots of attention in order to see which parts of an image
# our model focuses on  during captioning

import matplotlib.pyplot as plt

from pycocotools.coco import COCO

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

from tqdm import tqdm  # hadie
from termcolor import colored  # hadie
from builtins import len  # hadie
import datetime  # hadie

from model import DATASET, EXAMPLE_NUMBER, LIMIT_SIZE , WORD_DICT_SIZE, TEST_SET_PROPORTION, MY_EMBEDDING_DIM, UNIT_COUNT, CNN_Encoder, RNN_Decoder, MY_OPTIMIZER, MY_LOSS_OBJECT, REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN, EPOCH_COUNT, feature_extraction_model, split  # hadie
from timeit import default_timer as timer  # hadie
import threading  # hadie

import cv2
from transformers import AutoImageProcessor, ViTModel
import gc

# from yolo import image_path_to_yolo_bounding_boxes  # hadie

In [ ]:
# Hong
# 1. Load COCO training data

PATH ='/content/drive/MyDrive/coco/train2014/'
annotation_file = "/content/drive/MyDrive/coco/annotations/captions_train2014.json"

coco = COCO(annotation_file)
all_Ids = coco.getImgIds()
input_ids = shuffle(all_Ids, random_state=0)

num_examples = 20000
input_ids = input_ids[:num_examples]

input_img_name = []
for i in range(len(input_ids)):
    input_img_name.append(PATH+'COCO_train2014_' + '%012d' % (input_ids[i]) + '.jpg')

### input_data = {img_name_train[i]: [cap_1, cap_2, ... cap_5]}
input_data = {}
for i in range(0,len(input_img_name)):
    if input_img_name[i] not in input_data:
        annIds = coco.getAnnIds(imgIds = input_ids[i])
        anns = coco.loadAnns(annIds)
        captions = []
        for j in range(0,len(anns)):
            a = '<start> '+ anns[j]['caption'] + ' <end>'
            captions.append(a.lower())
        input_data[input_img_name[i]] = captions

# print(input_img_name[:5])
# print(input_ids[:5])

loading annotations into memory...
Done (t=3.08s)
creating index...
index created!


In [ ]:
############## FEATURES WITH MANUAL PREPROCESS ###############

# import torch
# import torchvision.transforms as transforms

# # image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
# vit_encoder = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
# for param in vit_encoder.parameters():
#     param.requires_grad = False

# total_params = sum(p.numel() for p in vit_encoder.parameters())
# print('Params of ViT:', total_params)

# IMAGE_SIZE = (224, 224)
# transform = transforms.Compose([
#     transforms.PILToTensor()
# ])

# def extract_vit_feature(filename):
#     image = Image.open(filename).convert("RGB")
#     image = image.resize(IMAGE_SIZE)

#     img_pytorch_tensor = transform(image)
#     img_pytorch_tensor = torch.unsqueeze(img_pytorch_tensor, 0)

#     pt_tensor_features = vit_encoder(img_pytorch_tensor).last_hidden_state

#     np_tensor_features = pt_tensor_features.numpy() # 1x197x768
#     features = np_tensor_features.reshape(-1, np_tensor_features.shape[2]) # 197x768

#     del image
#     del img_pytorch_tensor
#     del np_tensor_features
#     gc.collect()

#     return features

In [ ]:
############################################################
############## FEATURES WITH AUTO PREPROCESS ###############
############################################################

import torch
import torchvision.transforms as transforms

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
vit_encoder = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
for param in vit_encoder.parameters():
    param.requires_grad = False

total_params = sum(p.numel() for p in vit_encoder.parameters())
print('Params of ViT:', total_params)

def extract_vit_feature(filename):
    image = Image.open(filename).convert('RGB')
    inputs = image_processor(image, return_tensors="pt")

    with torch.no_grad():
        outputs = vit_encoder(**inputs)

    pt_tensor_features = outputs.last_hidden_state

    np_tensor_features = pt_tensor_features.numpy() # 1x197x768
    features = np_tensor_features.reshape(-1, np_tensor_features.shape[2]) # 197x768

    del image
    del inputs
    del outputs
    del pt_tensor_features
    del np_tensor_features
    gc.collect()

    return features

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Params of ViT: 86389248


In [ ]:
# extract combined feature for each image

print("-----------------------------START OF EXECUTION-----------------------------")  # hadie

feature_dir = './2_Vit_LSTM/ViT_npy_preprocess'

encode_train = [x for x in input_img_name if not os.path.exists(feature_dir + "/" + x[-31:-4] + ".npy")]
# features_shape = 2048
print("extracting features (" + str(len(encode_train)) + ") file(s)")
print('len of extracting',len (encode_train))

if len(encode_train) > 0:
    # print(encode_train)
    # Feel free to change batch_size according to your system configuration
    image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
    image_dataset = image_dataset.batch(16)

    for paths in tqdm(image_dataset):
        for path in paths:
            fn = path.numpy().decode("utf-8")
            combined_feature = extract_vit_feature(fn)
            np.save(feature_dir + "/" + fn[-31:-4], combined_feature)

            del fn
            del combined_feature
            gc.collect()

print("\nfinished extracting features")

-----------------------------START OF EXECUTION-----------------------------
extracting features (0) file(s)
len of extracting 0

finished extracting features


In [ ]:
"""## Preprocess and tokenize the captions

* First, you'll tokenize the captions (for example, by splitting on spaces).
This gives us a  vocabulary of all of the unique words in the data (for example, "surfing", "football", and so on).
* Next, you'll limit the vocabulary size to the top 5,000 words (to save memory).
You'll replace all other words with the token "UNK" (unknown).
* You then create word-to-index and index-to-word mappings.
* Finally, you pad all sequences to be the same length as the longest one.
"""
# Hong
def dict_to_list(train_descriptions):
    all_desc = []
    for key in train_descriptions.keys():
        [all_desc.append(d) for d in train_descriptions[key]]
    return all_desc

# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


# Choose the top 5000 words from the vocabulary
top_k = WORD_DICT_SIZE  # hadie

##################################################
REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN = False

input_captions = dict_to_list(input_data) # Hong
# print('input_captions',input_captions)

if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    print("using the cashed tokenizer")  # hadie
    # loading the tokenizer # hadie
    with open("./2_Vit_LSTM/model_trained_ViT_LSTM_att/tokenizer.pickle", 'rb') as handle:  # hadie
        tokenizer = pickle.load(handle)  # hadie
else:  # hadie
    print("tokenizing and padding captions")  # hadie
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                      oov_token="<unk>",
                                                      filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    # input_captions = dict_to_list(input_data) # Hong
    tokenizer.fit_on_texts(input_captions)

    # train_seqs = tokenizer.texts_to_sequences(input_captions)  # 777 maybe this line needs removal
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    # saving the tokenizer to disk # hadie
    with open("./2_Vit_LSTM/model_trained_ViT_LSTM_att/tokenizer.pickle", 'wb') as handle:  # hadie
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)  # hadie

# Create the tokenized vectors
# input_seqs = tokenizer.texts_to_sequences(input_captions)

# # Pad each vector to the max_length of the captions
# # If you do not provide a max_length value, pad_sequences calculates it automatically
# cap_vector = tf.keras.preprocessing.sequence.pad_sequences(input_seqs, padding='post')



tokenizing and padding captions


In [ ]:
# Create training and validation sets using an 90-10 split

PROP = 0.1

if PROP == 0:
    image_id_train = input_ids
    image_id_val = []
    img_name_train = input_img_name
    img_name_val = []
    # cap_train = cap_vector
    # cap_val = []
elif PROP == 1:
    image_id_train = []
    image_id_val = input_ids
    img_name_train = []
    img_name_val = input_img_name
    # cap_train = []
    # cap_val = cap_vector
else:
    image_id_train, image_id_val, img_name_train_2, img_name_val_2 = train_test_split(
                                                                    input_ids,  # hadie
                                                                    input_img_name,
                                                                    test_size=PROP,  # hadie
                                                                    random_state=0)

cap_train_list = []
cap_val_list = []
for i in range(len(img_name_train_2)):
    for j in range(len(input_data[img_name_train_2[i]])):
        cap_train_list.append(input_data[img_name_train_2[i]][j])
for i in range(len(img_name_val_2)):
    for j in range(len(input_data[img_name_val_2[i]])):
        cap_val_list.append(input_data[img_name_val_2[i]][j])

train_seqs = tokenizer.texts_to_sequences(cap_train_list)
val_seqs = tokenizer.texts_to_sequences(cap_val_list)
cap_train = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
cap_val = tf.keras.preprocessing.sequence.pad_sequences(val_seqs, padding='post')

# Hong
def dict_to_list_2(dict_data, list_key):
    all_img_name = []
    for i in range(len(list_key)):
        if list_key[i] in dict_data:
            for j in range(len(dict_data[list_key[i]])):
                all_img_name.append(list_key[i])
    return all_img_name

img_name_train = dict_to_list_2(input_data, img_name_train_2)
img_name_val = dict_to_list_2(input_data, img_name_val_2)

# print(img_name_train)
# print(cap_train_list)

# print(input_ids)
# print(input_img_name)
# print(cap_train)
# print("len(img_name_train) = ", len(img_name_train), ", len(cap_train) = ", len(cap_train), ", len(img_name_val) = ", len(img_name_val), ", len(cap_val) = ", len(cap_val))  # hadie

In [ ]:
if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    file = "./2_Vit_LSTM/model_trained_ViT_LSTM_att/max_length.txt"  # hadie
    with open(file, 'r') as filetoread:  # hadie
        max_length = int(filetoread.readline())  # hadie
else:  # hadie
    # Calculates the max_length, which is used to store the attention weights
    max_length = calc_max_length(train_seqs)

    # file = "trained_model_" + feature_extraction_model + "/max_length.txt"  # hadie
    file = "./2_Vit_LSTM/model_trained_ViT_LSTM_att/max_length.txt"  # hadie
    with open(file, 'w') as filetowrite:  # hadie
        filetowrite.write(str(max_length))  # write the maximum length to disk # hadie

print("finished tokenizing and padding captions")  # hadie

finished tokenizing and padding captions


In [ ]:
"""## Create a tf.data dataset for training

Our images and captions are ready! Next, let's create a tf.data dataset to use for training our model.
"""

# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 8*5  # 64
# BATCH_SIZE =16
# BUFFER_SIZE = BATCH_SIZE*8
embedding_dim = MY_EMBEDDING_DIM  # hadie
units = UNIT_COUNT  # hadie
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE
# Shape of the vector extracted from InceptionV3 is (64, 2048)
# These two variables represent that vector shape
# the definition of features_shape = 2048 was moved up
# the attention_features variable has been moved to a separate file by hadie


# Load the numpy files
def map_func(img_name, cap):
    img_tensor = np.load(feature_dir + '/' + img_name.decode('utf-8')[-31:-4] + '.npy')
    return img_tensor, cap

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))
dataset = dataset.shuffle(BATCH_SIZE*8)

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),
                        num_parallel_calls=tf.data.experimental.AUTOTUNE)

dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>


In [ ]:
"""## Model

"""

# the model has been moved to model.py by hadie

encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = MY_OPTIMIZER  # hadie
loss_object = MY_LOSS_OBJECT  # hadie


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


"""## Checkpoint"""

checkpoint_path = "./2_Vit_LSTM/checkpoint"

if REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:
    try:
        for filename in os.listdir(checkpoint_path):
            print("deleting " + checkpoint_path + "/" + filename)
            os.unlink(checkpoint_path + "/" + filename)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (checkpoint_path + "/" + filename, e))
    # remove the saved model too
    if os.path.exists("./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model.index"):
        print("deleting ./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model.index")
        os.unlink("./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model.index")
    if os.path.exists("./2_Vit_LSTM/model_trained_ViT_LSTM_att/checkpoint"):
        print("deleting ./2_Vit_LSTM/model_trained_ViT_LSTM_att/checkpoint")
        os.unlink("./2_Vit_LSTM/model_trained_ViT_LSTM_att/checkpoint")
    if os.path.exists("./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model.data-00000-of-00001"):
        print("deleting ./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model.data-00000-of-00001")
        os.unlink("./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model.data-00000-of-00001")
    if os.path.exists("./2_Vit_LSTM/model_trained_ViT_LSTM_att/learning_curve.png"):
        print("deleting ./2_Vit_LSTM/model_trained_ViT_LSTM_att/learning_curve.png")
        os.unlink("./2_Vit_LSTM/model_trained_ViT_LSTM_att/learning_curve.png")

ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    print('start_epoch', start_epoch)
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
"""## Training

"""

# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []


@tf.function
def train_step(img_tensor, target):
    loss = 0

    # initializing the hidden state for each batch
    # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)
        # print('encoder', encoder.count_params())

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)
            # print('decoder', decoder.count_params())

            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    del hidden
    del dec_input
    del features
    del predictions
    del trainable_variables
    gc.collect()

    return loss, total_loss


# EPOCHS = EPOCH_COUNT  # hadie
EPOCHS =10

if not os.path.exists("./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model.index"):
    print("training..")  # hadie

    t1 = time.time()
    for epoch in range(start_epoch, EPOCHS):
        start = time.time()
        total_loss = 0

        for (batch, (img_tensor, target)) in enumerate(dataset):
            # print('img_tensor.shape', img_tensor.shape)
            # print('target', target.shape)
            batch_loss, t_loss = train_step(img_tensor, target)
            total_loss += t_loss

            if batch % 100 == 0:
                print ('Epoch {} Batch {} Loss {:.4f}'.format(
                  epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
        # storing the epoch end loss value to plot later
        loss_plot.append(total_loss / num_steps)

        # if epoch % 5 == 0:
        ckpt_manager.save()

        print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                             total_loss / num_steps))
        print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    t2 = time.time()

    # save model to disk # hadie
    decoder.save_weights("./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model" , save_format="tf")  # hadie
    print('time of training: ', t2-t1)
else:  # hadie
    print("A trained model has been found. Loading it from disk..")  # hadie
    # Load the previously saved weights # hadie
    decoder.load_weights("./2_Vit_LSTM/model_trained_ViT_LSTM_att/my_model")  # hadie

training..
Epoch 1 Batch 0 Loss 2.0664
Epoch 1 Batch 100 Loss 1.2629
Epoch 1 Batch 200 Loss 1.1095
Epoch 1 Batch 300 Loss 0.9351
Epoch 1 Batch 400 Loss 0.9025
Epoch 1 Batch 500 Loss 0.8529
Epoch 1 Batch 600 Loss 0.8724
Epoch 1 Batch 700 Loss 0.8227
Epoch 1 Batch 800 Loss 0.7473
Epoch 1 Batch 900 Loss 0.8550
Epoch 1 Batch 1000 Loss 0.6806
Epoch 1 Batch 1100 Loss 0.7795
Epoch 1 Batch 1200 Loss 0.8872
Epoch 1 Batch 1300 Loss 0.7872
Epoch 1 Batch 1400 Loss 0.7702
Epoch 1 Batch 1500 Loss 0.7758
Epoch 1 Batch 1600 Loss 0.6875
Epoch 1 Batch 1700 Loss 0.7372
Epoch 1 Batch 1800 Loss 0.7443
Epoch 1 Batch 1900 Loss 0.7056
Epoch 1 Batch 2000 Loss 0.6797
Epoch 1 Batch 2100 Loss 0.7640
Epoch 1 Batch 2200 Loss 0.7098
Epoch 1 Loss 0.816579
Time taken for 1 epoch 2560.551245212555 sec

Epoch 2 Batch 0 Loss 0.6358
Epoch 2 Batch 100 Loss 0.6553
Epoch 2 Batch 200 Loss 0.6021
Epoch 2 Batch 300 Loss 0.6947
Epoch 2 Batch 400 Loss 0.7516
Epoch 2 Batch 500 Loss 0.6775
Epoch 2 Batch 600 Loss 0.6336
Epoch 2 Batc

In [ ]:
"""## Caption!

"""
attention_features_shape = 197

def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))
    # print(image)
    combined_features = extract_vit_feature(image)
    # print('combined_features.shape', combined_features.shape)
    features = encoder(combined_features)  # hadie

    hidden = decoder.reset_state(batch_size=1)
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)
        attention_plot[i] = tf.reshape(attention_weights, (-1,)).numpy()

        # predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        predicted_id = np.argmax(predictions.numpy())

        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result),:]
    return result, attention_plot


def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result // 2, len_result // 2, l + 1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

def list_to_dict(name_list, caption_list):
    name_dict = {}
    for i in range(len(name_list)):
        if name_list[i] not in name_dict:
            name_dict[name_list[i]] = [caption_list[i]]
        else:
            name_dict.get(name_list[i]).append(caption_list[i])
            # print( name_dict.get(name_list[i]))
    return name_dict


# captions on the validation set
val_dict = list_to_dict(img_name_val, cap_val)

from google.colab.patches import cv2_imshow

for i in range(10):
    # rid = np.random.randint(0, len(img_name_val))
    image = img_name_val[i*5]

    # real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
    captions = val_dict[image]

    result, attention_plot = evaluate(image)

    # print ('Real Caption:', real_caption)
    print('Real captions:')
    for j in range(len(captions)):
        real_caption = ' '.join([tokenizer.index_word[i] for i in captions[j] if i not in [0]])
        print(real_caption)

    print ('Prediction Caption:', ' '.join(result))
    # plot_attention(image, result, attention_plot)

    print('img path', image)

    img = cv2.imread(image)
    cv2_imshow(img)

# list_img = [
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000117179.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000335744.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000033435.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000555896.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000448069.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000367171.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000039758.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000192145.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000493483.jpg',
#     '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000519404.jpg'

# ]

# for image in list_img:
#     # real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
#     captions = val_dict[image]

#     result, attention_plot = evaluate(image)

#     # print ('Real Caption:', real_caption)
#     print('Real captions:')
#     for j in range(len(captions)):
#         real_caption = ' '.join([tokenizer.index_word[i] for i in captions[j] if i not in [0]])
#         print(real_caption)

#     print ('Prediction Caption:', ' '.join(result))
#     # plot_attention(image, result, attention_plot)

#     print('img path', image)

#     img = cv2.imread(image)
#     cv2_imshow(img)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


def img_to_caption(img_name):
    result, attention_plot = evaluate(img_name)
    result = ' '.join(result).replace("<end>", "").strip()
    return result

actual, predicted = [], []

for img_name in tqdm(val_dict):

    yhat = img_to_caption(img_name)
    pred = yhat.split()

    actu = []
    captions = val_dict[img_name]
    for i in range(len(captions)):
        real_caption = ' '.join([tokenizer.index_word[i] for i in captions[i] if i not in [0,3,4]])
        references = real_caption.split(' ')
        actu.append(references)

    actual.append(actu)
    predicted.append(pred)

bl1 = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0))
bl2 = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0))
bl3 = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0))
bl4 = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25))

mt = 0
for i in range(len(predicted)):
    mt += round(meteor_score(actual[i], predicted[i]),4)
mt = mt/len(predicted)

print("\nbleu-1 score:", bl1)
print("bleu-2 score:", bl2)
print("bleu-3 score:", bl3)
print("bleu-4 score:", bl4)
print("meteor score:", mt)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
100%|██████████| 2000/2000 [1:13:18<00:00,  2.20s/it]



bleu-1 score: 0.5759271554585981
bleu-2 score: 0.41538268949698104
bleu-3 score: 0.32449185708360434
bleu-4 score: 0.1929726389342297
meteor score: 0.4029793499999997


In [ ]:
# speed test

# for random samples
img_list = []
for i in range(100):
    rid = np.random.randint(0, len(img_name_val))
    img_list.append(img_name_val[rid])

t1 = time.time()
for image in img_list:
    result, attention_plot = evaluate(image)
    predict = ' '.join(result)

t2 = time.time()
print('time of predicting captions for 100 images:', t2-t1)

time of predicting captions for 100 images: 143.15592646598816


In [ ]:
# # speed test => moved to local test

# file = open('./save/img_name_val_bk.txt', 'r')
# txt = file.read()
# file.close()

# test_imgs1 = txt[1:-1].split(", ")
# test_imgs = []
# num_test_file = 5
# for i in range(num_test_file):
#     test_imgs.append(test_imgs1[i][1:-1])

# # ['/content/drive/MyDrive/coco/train2014/COCO_train2014_000000151458.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000419159.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000239584.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000521266.jpg',
# # '/content/drive/MyDrive/coco/train2014/COCO_train2014_000000337415.jpg']

# time_extract = 0
# time_predict = 0

# def predict_caption(image):

#     time_start_extract = time.time()
#     combined_features = extract_combined_feature(image)
#     t = time.time() - time_start_extract

#     features = encoder(combined_features)

#     hidden = decoder.reset_state(batch_size=1)
#     dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

#     result = []
#     for i in range(max_length):
#         predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

#         predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()

#         # hong
#         while predicted_id > len(tokenizer.index_word)-1:
#             predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
#         # hong end

#         result.append(tokenizer.index_word[predicted_id])

#         if tokenizer.index_word[predicted_id] == '<end>':
#             # print('early')
#             return result, t

#         dec_input = tf.expand_dims([predicted_id], 0)

#     return result, t


# # captions on the validation set
# n = len(test_imgs)
# for i in range(n):
#     image = test_imgs[i]

#     time_start_predict = time.time()
#     result, t = predict_caption(image)
#     time_predict += time.time() - time_start_predict
#     time_extract += t

#     print ('Prediction Caption:', ' '.join(result))

#     # print('img path', image)
#     # import cv2
#     # from google.colab.patches import cv2_imshow
#     # img = cv2.imread(image)
#     # cv2_imshow(img)

# time_extract /= n
# time_predict /= n
# print('average time_extract by Yolo_RNN', time_extract)
# print('average time_predict by Yolo_RNN', time_predict)